In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime

In [2]:
target_gu = '중랑구'
target_dong = '신내동'
#target_purpose = '준주거'
target_building_age = 5
target_period = 60 # in months

In [3]:
date_today = pd.to_datetime(datetime.datetime.now().strftime('%Y-%m-%d'), format='%Y-%m-%d')

In [4]:
date_today_record = str(date_today).split(' ')[0].replace('-', '')
date_today_record

'20210621'

In [5]:
def day_modifier(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if len(x) == 1:
            return '0' + x
        else:
            return x

In [6]:
def landnum_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x).replace('외', '').replace(' ','')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [ ]:
basedir = './국토교통부_실거래가_공개시스템/분양입주권/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.csv')]

In [13]:
def data_prep(bdtype, tradetype):
    # bdtype is a string: one of 아파트, 연립다세대 or 오피스텔
    # tradetype is a string: one of 매매 or 전월세
    
    basedir = './국토교통부_실거래가_공개시스템/{}/{}/'.format(bdtype, tradetype)
    if bdtype == '분양입주권':
        basedir = './국토교통부_실거래가_공개시스템/분양입주권/'
    filenames = [f for f in os.listdir(basedir) if f.endswith('.csv')]
    
    dfs_list = []
    for i, f in tqdm(enumerate(filenames)):
        df = pd.read_csv(basedir + f, encoding='euc-kr', header=15)
        if '해제사유발생일' in df.columns.tolist():
            df = df[df['해제사유발생일'].isna()]
            df = df.drop(columns=['해제사유발생일'])

        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    return concat_df
        
    concat_df['번지'] = concat_df['번지'].apply(landnum_modifier)
    
    concat_df['계약년월'] = concat_df['계약년월'].apply(str)
    concat_df['계약일'] = concat_df['계약일'].apply(str)
    
    concat_df['계약날짜기준_건물연식'] = concat_df['계약년월'].apply(lambda x: int(x[:4])) - concat_df['건축년도']
    
    concat_df['계약일'] = concat_df['계약일'].apply(day_modifier)
    
    concat_df['계약날짜'] = concat_df['계약년월'].apply(lambda x: x[:4]) + '-' + concat_df['계약년월'].apply(lambda x: x[-2:])\
    + '-' + concat_df['계약일']
    
    concat_df['계약날짜'] = pd.to_datetime(concat_df['계약날짜'], format='%Y-%m-%d')
    
    concat_df['거래금액(만원)'] = concat_df['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
    concat_df['단가(만원/㎡)'] = concat_df['거래금액(만원)'] / concat_df['전용면적(㎡)']
    
    concat_df['지번주소'] = concat_df['시군구'] + ' ' + concat_df['번지']
        
    cols_to_drop = ['시군구', '번지', '본번', '부번', '지번주소', '계약년월', '계약일', '도로명']
        
    concat_df = concat_df[['지번주소', '도로명'] + [col for col in concat_df.columns if col not in cols_to_drop]]
    
    date_today = pd.to_datetime(datetime.datetime.now().strftime('%Y-%m-%d'), format='%Y-%m-%d')
    concat_df['건물연식'] = date_today.year - concat_df['건축년도']
    
    concat_df = concat_df.dropna(subset=['지번주소'])
    
    if '건물명' in concat_df.columns:
        concat_df.rename(columns={'건물명':'건물(단지)명'}, inplace=True)
    elif '단지명' in concat_df.columns:
        concat_df.rename(columns={'단지명':'건물(단지)명'}, inplace=True)
    
    concat_df = concat_df[concat_df['층'] >= 0].reset_index(drop=True)
    
    return concat_df

In [14]:
sales_df = data_prep('분양입주권', None)
print(sales_df.shape)
sales_df.head()

16it [00:00, 123.05it/s]

(55011, 9)


,시군구,번지,단지명,전용면적(㎡),거래금액(만원),층,계약년월,계약일,분/입구분
0,서울특별시 강남구 삼성동,16,삼성동 현대아파트(영동차관),84.236,"95,500",12,200707,3,입
1,서울특별시 강남구 삼성동,16,삼성동 현대아파트(영동차관),84.236,"96,000",13,200707,11,입
2,서울특별시 강남구 삼성동,16,삼성동 현대아파트(영동차관),84.236,"90,000",2,200707,14,입
3,서울특별시 강남구 삼성동,16,삼성동 현대아파트(영동차관),84.236,"75,000",1,200707,18,입
4,서울특별시 강남구 삼성동,16,삼성동 현대아파트(영동차관),84.236,"95,000",9,200708,26,입


In [15]:
sales_df['지번주소'] = sales_df['시군구'] + ' ' + sales_df['번지']

In [16]:
sales_df.head()

,시군구,번지,단지명,전용면적(㎡),거래금액(만원),층,계약년월,계약일,분/입구분,지번주소
0,서울특별시 강남구 삼성동,16,삼성동 현대아파트(영동차관),84.236,"95,500",12,200707,3,입,서울특별시 강남구 삼성동 16
1,서울특별시 강남구 삼성동,16,삼성동 현대아파트(영동차관),84.236,"96,000",13,200707,11,입,서울특별시 강남구 삼성동 16
2,서울특별시 강남구 삼성동,16,삼성동 현대아파트(영동차관),84.236,"90,000",2,200707,14,입,서울특별시 강남구 삼성동 16
3,서울특별시 강남구 삼성동,16,삼성동 현대아파트(영동차관),84.236,"75,000",1,200707,18,입,서울특별시 강남구 삼성동 16
4,서울특별시 강남구 삼성동,16,삼성동 현대아파트(영동차관),84.236,"95,000",9,200708,26,입,서울특별시 강남구 삼성동 16


In [17]:
sales_df = sales_df.sort_values(['지번주소']).reset_index(drop=True)

In [18]:
sales_df.head()

,시군구,번지,단지명,전용면적(㎡),거래금액(만원),층,계약년월,계약일,분/입구분,지번주소
0,서울특별시 강남구 개포동,1280,래미안블레스티지,84.943,"176,000",3,201905,17,분,서울특별시 강남구 개포동 1280
1,서울특별시 강남구 개포동,1280,래미안블레스티지,99.991,"205,000",10,201904,17,분,서울특별시 강남구 개포동 1280
2,서울특별시 강남구 개포동,1280,래미안블레스티지,49.909,"115,000",5,201904,15,분,서울특별시 강남구 개포동 1280
3,서울특별시 강남구 개포동,1280,래미안블레스티지,126.928,"255,000",11,201903,23,분,서울특별시 강남구 개포동 1280
4,서울특별시 강남구 개포동,1280,래미안블레스티지,59.967,"144,000",8,201903,14,입,서울특별시 강남구 개포동 1280


In [19]:
first_df = sales_df[sales_df['지번주소'] == sales_df['지번주소'].unique()[0]]
print(first_df.shape)
first_df.head()

(6, 10)


,시군구,번지,단지명,전용면적(㎡),거래금액(만원),층,계약년월,계약일,분/입구분,지번주소
0,서울특별시 강남구 개포동,1280,래미안블레스티지,84.943,"176,000",3,201905,17,분,서울특별시 강남구 개포동 1280
1,서울특별시 강남구 개포동,1280,래미안블레스티지,99.991,"205,000",10,201904,17,분,서울특별시 강남구 개포동 1280
2,서울특별시 강남구 개포동,1280,래미안블레스티지,49.909,"115,000",5,201904,15,분,서울특별시 강남구 개포동 1280
3,서울특별시 강남구 개포동,1280,래미안블레스티지,126.928,"255,000",11,201903,23,분,서울특별시 강남구 개포동 1280
4,서울특별시 강남구 개포동,1280,래미안블레스티지,59.967,"144,000",8,201903,14,입,서울특별시 강남구 개포동 1280


In [20]:
first_df

,시군구,번지,단지명,전용면적(㎡),거래금액(만원),층,계약년월,계약일,분/입구분,지번주소
0,서울특별시 강남구 개포동,1280,래미안블레스티지,84.943,"176,000",3,201905,17,분,서울특별시 강남구 개포동 1280
1,서울특별시 강남구 개포동,1280,래미안블레스티지,99.991,"205,000",10,201904,17,분,서울특별시 강남구 개포동 1280
2,서울특별시 강남구 개포동,1280,래미안블레스티지,49.909,"115,000",5,201904,15,분,서울특별시 강남구 개포동 1280
3,서울특별시 강남구 개포동,1280,래미안블레스티지,126.928,"255,000",11,201903,23,분,서울특별시 강남구 개포동 1280
4,서울특별시 강남구 개포동,1280,래미안블레스티지,59.967,"144,000",8,201903,14,입,서울특별시 강남구 개포동 1280
5,서울특별시 강남구 개포동,1280,래미안블레스티지,126.928,"247,360",4,201903,12,분,서울특별시 강남구 개포동 1280


In [22]:
sales_df.isna().sum()

시군구          0
번지          48
단지명          0
전용면적(㎡)      0
거래금액(만원)     0
층            0
계약년월         0
계약일          0
분/입구분        4
지번주소        48
dtype: int64

In [23]:
sales_df.dropna(inplace=True)

In [25]:
inv_df = sales_df[sales_df['지번주소'].str.contains('중랑구 신내동')].sort_values(['지번주소'])
print(inv_df.shape)
inv_df.head()

(163, 10)


,시군구,번지,단지명,전용면적(㎡),거래금액(만원),층,계약년월,계약일,분/입구분,지번주소
54774,서울특별시 중랑구 신내동,317,데시앙,84.82,"28,692",2,200912,4,분,서울특별시 중랑구 신내동 317
54870,서울특별시 중랑구 신내동,317,데시앙,114.71,"42,256",2,201004,29,분,서울특별시 중랑구 신내동 317
54871,서울특별시 중랑구 신내동,317,데시앙,114.48,"44,285",13,201004,16,분,서울특별시 중랑구 신내동 317
54872,서울특별시 중랑구 신내동,317,데시앙,114.71,"41,105",1,201004,7,분,서울특별시 중랑구 신내동 317
54873,서울특별시 중랑구 신내동,317,데시앙,114.71,"42,583",14,201003,26,분,서울특별시 중랑구 신내동 317


In [26]:
inv_df

,시군구,번지,단지명,전용면적(㎡),거래금액(만원),층,계약년월,계약일,분/입구분,지번주소
54774,서울특별시 중랑구 신내동,317,데시앙,84.82,"28,692",2,200912,4,분,서울특별시 중랑구 신내동 317
54870,서울특별시 중랑구 신내동,317,데시앙,114.71,"42,256",2,201004,29,분,서울특별시 중랑구 신내동 317
54871,서울특별시 중랑구 신내동,317,데시앙,114.48,"44,285",13,201004,16,분,서울특별시 중랑구 신내동 317
54872,서울특별시 중랑구 신내동,317,데시앙,114.71,"41,105",1,201004,7,분,서울특별시 중랑구 신내동 317
54873,서울특별시 중랑구 신내동,317,데시앙,114.71,"42,583",14,201003,26,분,서울특별시 중랑구 신내동 317
54874,서울특별시 중랑구 신내동,317,데시앙,114.71,"40,881",3,201003,26,분,서울특별시 중랑구 신내동 317
54875,서울특별시 중랑구 신내동,317,데시앙,114.71,"42,583",13,201003,26,분,서울특별시 중랑구 신내동 317
54869,서울특별시 중랑구 신내동,317,데시앙,114.71,"42,583",14,201003,25,분,서울특별시 중랑구 신내동 317
54876,서울특별시 중랑구 신내동,317,데시앙,114.71,"42,583",12,201003,26,분,서울특별시 중랑구 신내동 317
54878,서울특별시 중랑구 신내동,317,데시앙,114.71,"42,157",8,201003,25,분,서울특별시 중랑구 신내동 317
